<a href="https://colab.research.google.com/github/akansh12/CS6910_Assignment/blob/main/partB/Part_B_Akansh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import pandas as pd
import numpy as np
import torchvision
import os
import torch.nn as nn

import matplotlib.pyplot as plt
import copy
from tqdm.notebook import tqdm

In [6]:
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
path2data_train="/content/drive/MyDrive/Nature_data/inaturalist_12K/train"
path2data_test = "/content/drive/MyDrive/Nature_data/inaturalist_12K/val"

In [7]:
train_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.RandomRotation((-20,20)),
transforms.RandomHorizontalFlip(p=0.5),
transforms.RandomVerticalFlip(p=0.5),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [26]:
data = datasets.ImageFolder(path2data_train, train_transforms)
test_data = datasets.ImageFolder(path2data_test, test_transforms)
n_val = int(np.floor(0.1 * len(data)))
n_train = len(data) - n_val
train_ds, val_ds = random_split(data, [n_train, n_val])

print("Number of datapoints in train: ", len(train_ds))
print("Number of datapoints in val: ", len(val_ds))
print("Number of datapoints in test: ", len(test_data))

Number of datapoints in train:  9000
Number of datapoints in val:  999
Number of datapoints in test:  2000


In [ ]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=False)
test_dl = DataLoader(test_data, batch_size=64, shuffle=False)

### Question 1

I will be experimenting with 
- InceptionV3, 
- DenseNet121
- ResNet50

Others can be added to the code.

In [9]:
def model_define(name= "resnet50", num_classes = 10):
  if name == "resnet50":
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(2048, num_classes)
  if name == "inceptionv3":
    model = models.inception_v3(pretrained = True)
    model.fc = nn.Linear(2048, 10)
  if name == "densenet121":
    model = models.densenet121(pretrained=True)
    model.classifier = nn.Linear(1024, 10)
  return model

In [21]:
model = model_define("densenet121")

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [28]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

### Answer

1. The dimensions of the images in your data may not be the same as that in the ImageNet data. How will you address this?
- ANS: The image size of ImageNET dataset is (224, 224, 3), while peforming augmentation, I will add Resize transform layer in my code to change all the input image of iNaturalist to 224,224,3

2. ImageNet has 1000 classes and hence the last layer of the pre-trained model would have 1000 nodes. However, the naturalist dataset has only 10 classes. How will you address this?

- ANS: I have changed the last fully connected layer with neurons equal to the number of classes, in this case it is 10. 